<a href="https://colab.research.google.com/github/wwan0255/PETalk/blob/main/FIT5215_DeepLearning_Assignment2_Official%5BTransformers%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="#0b486b">  FIT5215: Deep Learning (2025) - Assignment 2 (Section II)</font>
***
*CE/Lecturer (Clayton):*  **Dr Trung Le** | trunglm@monash.edu <br/>
*Lecturer (Clayton):* **A/Prof Zongyuan Ge** | zongyuan.ge@monash.edu <br/>
*Lecturer (Malaysia):*  **Dr Arghya Pal** | arghya.pal@monash.edu <br/>
 <br/>
*Head Tutor 3181:*  **Ms Ruda Nie H** |  \[RudaNie.H@monash.edu \] <br/>
*Head Tutor 5215:*  **Ms Leila Mahmoodi** |  \[leila.mahmoodi@monash.edu \]

<br/> <br/>
Faculty of Information Technology, Monash University, Australia
***

# <font color="#0b486b">  Student Information</font>
***
Surname: **[Wang]**  <br/>
Firstname: **[Shuaiqi]**    <br/>
Student ID: **[35630418]**    <br/>
Email: **[wwan0255@student.monash.edu]**    <br/>
Your tutorial time: **[12-2pm，Wednesday]**    <br/>
***

# <font color="0b486b">Assignment 2 – Deep Learning for Sequential Data</font>
### Due: <font color="red">11:55pm Sunday, 26 October 2025</font> (FIT5215)

#### <font color="red">Important note:</font> This is an **individual** assignment. It contributes **20%** to your final mark. Read the assignment instructions carefully.

## <font color="#0b486b">Assignment 2's Organization</font>
This assignment 2 has two (2) sections:
- Section 1: Fundamentals of RNNs (10 marks).
- Section 2: Deep Learning for Sequential Data (90 marks). This section is further divided into 4 parts.

The assignment 2 is organized in three (3) notebooks.
- Notebook 1 ([link](https://colab.research.google.com/drive/1hTS6bWz6gU-lvi1qkIhrtQnJ3tzxRPFK?usp=sharing
)) [Total: 30 marks] includes Section 1 as well as Part 1 and Part 2 of Section 2.
- Notebook 2 ([link](https://colab.research.google.com/drive/1m0mh9Mk4-AKEhgAHRwQdl5mc0x7SF7Tv?usp=sharing)) [Total: 40 marks] includes Part 3 of Section 2.
- Notebook 3 (this notebook) [Total: 30 marks] includes Part 4 of Section 2.


## <font color="#0b486b">What to submit</font>

This assignment is to be completed individually and submitted to Moodle unit site. **By the due date, you are required to submit one  <font color="red; font-weight:bold">single zip file, named xxx_assignment02_solution.zip</font> where `xxx` is your student ID, to the corresponding Assignment (Dropbox) in Moodle**. You can use Google Colab to do Assignment 2 but you need to save it to an `*.ipynb` file to submit to the unit Moodle.

**More importantly, if you use Google Colab to do this assignment, you need to first make a copy of this notebook on your Google drive**.

***For example, if your student ID is <font color="red; font-weight:bold">12356</font>, then gather all of your assignment solutions to a folder, create a zip file named <font color="red; font-weight:bold">123456_assignment02_solution.zip</font> and submit this file.***

Within this zip folder, you **must** submit the following files <u>for each part</u>:
1.	**`FIT5215_DeepLearning_Assignment2_Official[Main].ipynb`**:  this is your Python notebook solution source file.
1.	**`FIT5215_DeepLearning_Assignment2_Official[Main].html`**: this is the output of your Python notebook solution *exported* in HTML format.
1. **`FIT5215_DeepLearning_Assignment2_Official[RNNs].ipynb`**
1. **`FIT5215_DeepLearning_Assignment2_Official[RNNs].html`**
1. **`FIT5215_DeepLearning_Assignment2_Official[Transformers].ipynb`**
1. **`FIT5215_DeepLearning_Assignment2_Official[Transformers].html`**
1.	Any **extra files or folder** needed to complete your assignment (e.g., images used in your answers).



## Section 2: Deep Learning for Sequential Data

### <font color="#0b486b">Set random seeds</font>

We need to install the package datasets for creating BERT datasets.

In [6]:
!pip install datasets

We start with importing PyTorch and NumPy and setting random seeds for PyTorch and NumPy. You can use any seeds you prefer.

In [15]:
import os
import torch
import random
import requests
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer
import os
from six.moves.urllib.request import urlretrieve
from sklearn import preprocessing
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [16]:
def seed_all(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_all(seed=1234)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## <font color="#0b486b">Download and preprocess the data</font>

<div style="text-align: right"><font color="red; font-weight:bold"><span></div>

The dataset we use for this assignment is a question classification dataset for which the training set consists of $5,500$ questions belonging to 6 coarse question categories including:
- abbreviation (ABBR),
- entity (ENTY),
- description (DESC),
- human (HUM),
- location (LOC) and
- numeric (NUM).

In this assignment, we will utilize a subset of this dataset, containing $2,000$ questions for training and validation. We will use 80% of those 2000 questions for trainning and the rest for validation.


Preprocessing data is a crucial initial step in any machine learning or deep learning project. The *TextDataManager* class simplifies the process by providing functionalities to download and preprocess data specifically designed for the subsequent questions in this assignment. It is highly recommended to gain a comprehensive understanding of the class's functionality by **carefully reading** the content provided in the *TextDataManager* class before proceeding to answer the questions.

In [17]:
class DataManager:
    """
    This class manages and preprocesses a simple text dataset for a sentence classification task.

    Attributes:
        verbose (bool): Controls verbosity for printing information during data processing.
        max_sentence_len (int): The maximum length of a sentence in the dataset.
        str_questions (list): A list to store the string representations of the questions in the dataset.
        str_labels (list): A list to store the string representations of the labels in the dataset.
        numeral_labels (list): A list to store the numerical representations of the labels in the dataset.
        maxlen (int): Maximum length for padding sequences. Sequences longer than this length will be truncated,
            and sequences shorter than this length will be padded with zeros. Defaults to 50.
        numeral_data (list): A list to store the numerical representations of the questions in the dataset.
        random_state (int): Seed value for random number generation to ensure reproducibility.
            Set this value to a specific integer to reproduce the same random sequence every time. Defaults to 6789.
        random (np.random.RandomState): Random number generator object initialized with the given random_state.
            It is used for various random operations in the class.

    Methods:
        maybe_download(dir_name, file_name, url, verbose=True):
            Downloads a file from a given URL if it does not exist in the specified directory.
            The directory and file are created if they do not exist.

        read_data(dir_name, file_names):
            Reads data from files in a directory, preprocesses it, and computes the maximum sentence length.
            Each file is expected to contain rows in the format "<label>:<question>".
            The labels and questions are stored as string representations.

        manipulate_data():
            Performs data manipulation by tokenizing, numericalizing, and padding the text data.
            The questions are tokenized and converted into numerical sequences using a tokenizer.
            The sequences are padded or truncated to the maximum sequence length.

        train_valid_test_split(train_ratio=0.9):
            Splits the data into training, validation, and test sets based on a given ratio.
            The data is randomly shuffled, and the specified ratio is used to determine the size of the training set.
            The string questions, numerical data, and numerical labels are split accordingly.
            TensorFlow `Dataset` objects are created for the training and validation sets.


    """

    def __init__(self, verbose=True, random_state=6789):
        self.verbose = verbose
        self.max_sentence_len = 0
        self.str_questions = list()
        self.str_labels = list()
        self.numeral_labels = list()
        self.numeral_data = list()
        self.random_state = random_state
        self.random = np.random.RandomState(random_state)

    @staticmethod
    def maybe_download(dir_name, file_name, url, verbose=True):
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        if not os.path.exists(os.path.join(dir_name, file_name)):
            urlretrieve(url + file_name, os.path.join(dir_name, file_name))
        if verbose:
            print("Downloaded successfully {}".format(file_name))

    def read_data(self, dir_name, file_names):
        self.str_questions = list()
        self.str_labels = list()
        for file_name in file_names:
            file_path= os.path.join(dir_name, file_name)
            with open(file_path, "r", encoding="latin-1") as f:
                for row in f:
                    row_str = row.split(":")
                    label, question = row_str[0], row_str[1]
                    question = question.lower()
                    self.str_labels.append(label)
                    self.str_questions.append(question[0:-1])
                    if self.max_sentence_len < len(self.str_questions[-1]):
                        self.max_sentence_len = len(self.str_questions[-1])

        # turns labels into numbers
        le = preprocessing.LabelEncoder()
        le.fit(self.str_labels)
        self.numeral_labels = np.array(le.transform(self.str_labels))
        self.str_classes = le.classes_
        self.num_classes = len(self.str_classes)
        if self.verbose:
            print("\nSample questions and corresponding labels... \n")
            print(self.str_questions[0:5])
            print(self.str_labels[0:5])

    def manipulate_data(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        vocab = self.tokenizer.get_vocab()
        self.word2idx = {w: i for i, w in enumerate(vocab)}
        self.idx2word = {i:w for w,i in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)

        token_ids = []
        num_seqs = []
        for text in self.str_questions:  # iterate over the list of text
          text_seqs = self.tokenizer.tokenize(str(text))  # tokenize each text individually
          # Convert tokens to IDs
          token_ids = self.tokenizer.convert_tokens_to_ids(text_seqs)
          # Convert token IDs to a tensor of indices using your word2idx mapping
          seq_tensor = torch.LongTensor(token_ids)
          num_seqs.append(seq_tensor)  # append the tensor for each sequence

        # Pad the sequences and create a tensor
        if num_seqs:
          self.numeral_data = pad_sequence(num_seqs, batch_first=True)  # Pads to max length of the sequences
          self.num_sentences, self.max_seq_len = self.numeral_data.shape


    def train_valid_test_split(self, train_ratio=0.8, test_ratio = 0.1):
        num_sentences = self.numeral_data.shape[0]
        train_size = int(num_sentences*train_ratio) +1
        test_size = int(num_sentences*test_ratio) +1
        valid_size = num_sentences - (train_size + test_size)
        data_indices = list(range(num_sentences))
        random.shuffle(data_indices)

        train_set_data = self.numeral_data[data_indices[:train_size]]
        train_set_labels = self.numeral_labels[data_indices[:train_size]]
        train_set_labels = torch.from_numpy(train_set_labels)
        train_set = torch.utils.data.TensorDataset(train_set_data, train_set_labels)

        test_set_data = self.numeral_data[data_indices[-test_size:]]
        test_set_labels = self.numeral_labels[data_indices[-test_size:]]
        test_set_labels = torch.from_numpy(test_set_labels)
        test_set = torch.utils.data.TensorDataset(test_set_data, test_set_labels)

        valid_set_data = self.numeral_data[data_indices[train_size:-test_size]]
        valid_set_labels = self.numeral_labels[data_indices[train_size:-test_size]]
        valid_set_labels = torch.from_numpy(valid_set_labels)
        valid_set = torch.utils.data.TensorDataset(valid_set_data, valid_set_labels)

        self.train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
        self.test_loader = DataLoader(test_set, batch_size=64, shuffle=False)
        self.valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False)

In [18]:
print('Loading data...')
DataManager.maybe_download("data", "train_2000.label", "http://cogcomp.org/Data/QA/QC/")

dm = DataManager()
dm.read_data("data/", ["train_2000.label"])

Loading data...
Downloaded successfully train_2000.label

Sample questions and corresponding labels... 

['manner how did serfdom develop in and then leave russia ?', 'cremat what films featured the character popeye doyle ?', "manner how can i find a list of celebrities ' real names ?", 'animal what fowl grabs the spotlight after the chinese year of the monkey ?', 'exp what is the full form of .com ?']
['DESC', 'ENTY', 'DESC', 'ENTY', 'ABBR']


In [19]:
dm.manipulate_data()
dm.train_valid_test_split(train_ratio=0.8, test_ratio = 0.1)

In [20]:
for x, y in dm.train_loader:
    print(x.shape, y.shape)
    break

torch.Size([64, 36]) torch.Size([64])


We now declare the `BaseTrainer` class, which will be used later to train the subsequent deep learning models for text data.

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class BaseTrainer:
    def __init__(self, model, criterion, optimizer, train_loader, val_loader):
        self.model = model
        self.criterion = criterion  #the loss function
        self.optimizer = optimizer  #the optimizer
        self.train_loader = train_loader  #the train loader
        self.val_loader = val_loader  #the valid loader

    #the function to train the model in many epochs
    def fit(self, num_epochs):
        self.num_batches = len(self.train_loader)

        for epoch in range(num_epochs):
            print(f'Epoch {epoch + 1}/{num_epochs}')
            train_loss, train_accuracy = self.train_one_epoch()
            val_loss, val_accuracy = self.validate_one_epoch()
            print(
                f'{self.num_batches}/{self.num_batches} - train_loss: {train_loss:.4f} - train_accuracy: {train_accuracy*100:.4f}% \
                - val_loss: {val_loss:.4f} - val_accuracy: {val_accuracy*100:.4f}%')

    #train in one epoch, return the train_acc, train_loss
    def train_one_epoch(self):
        self.model.train()
        running_loss, correct, total = 0.0, 0, 0
        for i, data in enumerate(self.train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_accuracy = correct / total
        train_loss = running_loss / self.num_batches
        return train_loss, train_accuracy

    #evaluate on a loader and return the loss and accuracy
    def evaluate(self, loader):
        self.model.eval()
        loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for data in loader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        loss = loss / len(self.val_loader)
        return loss, accuracy

    #return the val_acc, val_loss, be called at the end of each epoch
    def validate_one_epoch(self):
      val_loss, val_accuracy = self.evaluate(self.val_loader)
      return val_loss, val_accuracy

## <font color="#0b486b">Part 4: Transformer-based models for sequence modeling and neural embedding</font>

<div style="text-align: right"><font color="red; font-weight:bold">[Total marks for this part: 30 marks]<span></div>

#### <font color="red">**Question 4.1**</font>

**Implement the multi-head attention module of the Transformer for the text classification problem. The provided code is from our tutorial. In this part, we only use the output of the Transformer encoder for the classification task. For further information on the Transformer model, refer to [this paper](https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf).**

<div style="text-align: right"><font color="red; font-weight:bold">[Total marks for this part: 10 marks]<span></div>


Below is the code of `MultiHeadSelfAttention`, `PositionWiseFeedForward`, `PositionalEncoding`, and `EncoderLayer`.

In [23]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value

        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation

    def scaled_dot_product_attention(self, Q, K, V):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        #if mask is not None:
            #attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V)

        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [24]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [25]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [30]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output = self.self_attn(x, x, x)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

Your task is to develop `TransformerClassifier` in which we input the embedding with the shape `[batch_size, seq_len, embed_dim]` to some `EncoderLayer` (i.e., num_layers specifies the number of EncoderLayer) and then compute the average of all token embeddings (i.e., `[batch_size, seq_len, embed_dim]`) across the `seq_len`. Finally, on the top of this average embedding, we build up a linear layer for making predictions.

In [26]:
class TransformerClassifier(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, num_layers, dropout_rate=0.2, data_manager = None):
        super(TransformerClassifier, self).__init__()
        self.vocab_size = data_manager.vocab_size
        self.num_classes = data_manager.num_classes
        self.embed_dim = embed_dim
        self.max_seq_len = data_manager.max_seq_len
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate

        self.embedding = nn.Embedding(self.vocab_size, self.embed_dim)
        self.positional_encoding = PositionalEncoding(self.embed_dim, self.max_seq_len)
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(self.embed_dim, self.num_heads, self.ff_dim, self.dropout_rate)
            for _ in range(self.num_layers)
        ])
        self.dropout = nn.Dropout(self.dropout_rate)
        self.final_layer = nn.Linear(self.embed_dim, self.num_classes)


    def forward(self, x):
        # ids -> embeddings
        x = self.embedding(x)
        # add positions
        x = self.positional_encoding(x)
        # dropout regularization
        x = self.dropout(x)
        # stacked Transformer encoders
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        # mean-pool over time
        x = torch.mean(x, dim=1)
        # logits (no softmax; CrossEntropyLoss expects logits)
        x = self.final_layer(x)
        return x

In [31]:
transformer = TransformerClassifier(embed_dim=512, num_heads=8, ff_dim=2048, num_layers=6, dropout_rate=0.1, data_manager= dm)
transformer = transformer.to(device)
print(transformer)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transformer.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)
trainer = BaseTrainer(model= transformer, criterion=criterion, optimizer=optimizer, train_loader=dm.train_loader, val_loader=dm.valid_loader)
trainer.fit(num_epochs=30)

TransformerClassifier(
  (embedding): Embedding(30522, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (final_layer): Linear(in_features=512, out_features=6, bia

In [32]:
# Evaluate the best model
# Use the test loader from the DataManager
test_loss, test_accuracy = trainer.evaluate(dm.test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.2452, Test Accuracy: 0.9502


In [33]:
import torch

# Define the file name for your best model
MODEL_SAVE_PATH = 'best_transformer_model.pth'

torch.save(transformer.state_dict(), MODEL_SAVE_PATH)

print(f"✅ Model 'transformer' successfully saved as '{MODEL_SAVE_PATH}'.")


✅ Model 'transformer' successfully saved as 'best_transformer_model.pth'.


#### <font color="red">**Question 4.2**</font>
**Prefix prompt-tuning with Transformers: You need to implement the prefix prompt-tuning with Transformers. Basically, we base on a pre-trained Transformer, add prefix prompts, and do fine-tuning for a target dataset.**

<div style="text-align: right"><font color="red; font-weight:bold">[Total marks for this part: 10 marks]<span></div>

To implement prefix prompt-tuning with pretrained Transformers, we first need to create the Bert dataset.

In [35]:
!pip install --upgrade datasets transformers

In [36]:
from transformers import AutoModel, AutoTokenizer
from datasets import Dataset

model_name = "bert-base-uncased"  # BERT or any similar model

# Tokenize input and prepare model inputs
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = Dataset.from_dict({"text": dm.str_questions, "label": dm.numeral_labels})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length= 36)

dataset = dataset.map(tokenize_function, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
print(dataset)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})


The following function splits the BERT dataset `dataset` into three BERT datasets for training, valid, and testing.

In [37]:
def train_valid_test_split(dataset, train_ratio=0.8, test_ratio = 0.1):
    num_sentences = len(dataset)
    train_size = int(num_sentences*train_ratio) +1
    test_size = int(num_sentences*test_ratio) +1
    valid_size = num_sentences - (train_size + test_size)
    train_set = dataset[:train_size]
    train_set = Dataset.from_dict(train_set)
    train_set.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    test_set = dataset[-test_size:]
    test_set = Dataset.from_dict(test_set)
    test_set.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    valid_set = dataset[train_size:-test_size]
    valid_set = Dataset.from_dict(valid_set)
    valid_set.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=64, shuffle=False)
    valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False)
    return train_loader, test_loader, valid_loader


In [38]:
train_loader, test_loader, valid_loader = train_valid_test_split(dataset)

You need to implement the class `PrefixTuningForClassification` for the prefix prompt fine-tuning. We first load a pre-trained BERT model specified by `model_name`. The parameter `prefix_length` specifies the length of the prefix prompts we add to the pre-trained BERT model. Specifically, given the input batch `[batch_size, seq_len]`, we input to the embedding layer of the pre-trained BERT model to obtain `[batch_size, seq_len, embed_size]`. We create the prefix prompts $P$ of the size `[prefix_length, embed_size]` and concatenate to the embeddings from the pre-trained BERT to obtain `[batch_size, seq_len + prefix_length, embed_size]`. This concatenation tensor will then be fed to the encoder layers of the pre-trained BERT layer to obtain the last `[batch_size, seq_len + prefix_length, embed_size]`.

We then take mean across the seq_len to obtain `[batch_size, embed_size]` on which we can build up a linear layer for making predictions. Please note that **the parameters to tune include the prefix prompts $P$** and **the output linear layer**, and you should freeze the parameters of the BERT pre-trained model. Moreover, your code should cover the edge case when `prefix_length=None`. In this case, we do not insert any prefix prompts and we only do fine-tuning for the output linear layer on top.  

In [39]:
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from datasets import Dataset

class PrefixTuningForClassification(nn.Module):
    def __init__(self, model_name, prefix_length=None, data_manager=None):
        super().__init__()
        # load pretrained transformer backbone (BERT-like) and move to device
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.hidden_size = self.model.config.hidden_size
        self.prefix_length = prefix_length
        self.num_classes = data_manager.num_classes

        # freeze backbone parameters (we only fine-tune prefix + classifier)
        for param in self.model.parameters():
            param.requires_grad = False

        # optional learnable prefix embeddings (prefix-tuning)
        if self.prefix_length is not None:
            # shape: (prefix_length, hidden_size)
            self.prefix_embeddings = nn.Parameter(torch.randn(self.prefix_length, self.hidden_size))
            nn.init.xavier_uniform_(self.prefix_embeddings)

        # classification head mapping pooled hidden -> class logits
        self.classifier = nn.Linear(self.hidden_size, self.num_classes)

        self.device = device
        self.to(self.device)


    def forward(self, input_ids, attention_mask):
        """
        input_ids: (B, T) token ids
        attention_mask: (B, T) attention mask for the tokens
        """
        batch_size, _ = input_ids.shape

        if self.prefix_length is not None:
            # expand prefix to batch and concatenate with token embeddings
            prefix = self.prefix_embeddings.unsqueeze(0).expand(batch_size, -1, -1).to(self.device)
            # get token embeddings from the backbone embedding layer
            token_embeds = self.model.embeddings.word_embeddings(input_ids)
            # combined embeddings: [prefix_tokens, original_tokens]
            combined_embeddings = torch.cat([prefix, token_embeds], dim=1)

            # extend attention mask to cover prefix tokens (1 = attendable)
            prefix_mask = torch.ones(batch_size, self.prefix_length, device=self.device)
            combined_attention_mask = torch.cat([prefix_mask, attention_mask], dim=1)

            # run the model using inputs_embeds + combined mask
            outputs = self.model(inputs_embeds=combined_embeddings,
                                 attention_mask=combined_attention_mask)
        else:
            # standard forward using input ids and mask
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        # pool the last hidden states by mean across time -> (B, H)
        pooled_output = outputs.last_hidden_state.mean(dim=1)

        # classifier -> logits (CrossEntropyLoss expects raw logits)
        logits = self.classifier(pooled_output)
        return logits

You can use the following `FineTunedBaseTrainer` to train the prompt fine-tuning models.

In [40]:
class FineTunedBaseTrainer:
    def __init__(self, model, criterion, optimizer, train_loader, val_loader):
        self.model = model
        self.criterion = criterion  #the loss function
        self.optimizer = optimizer  #the optimizer
        self.train_loader = train_loader  #the train loader
        self.val_loader = val_loader  #the valid loader

    #the function to train the model in many epochs
    def fit(self, num_epochs):
        self.num_batches = len(self.train_loader)

        for epoch in range(num_epochs):
            print(f'Epoch {epoch + 1}/{num_epochs}')
            train_loss, train_accuracy = self.train_one_epoch()
            val_loss, val_accuracy = self.validate_one_epoch()
            print(
                f'{self.num_batches}/{self.num_batches} - train_loss: {train_loss:.4f} - train_accuracy: {train_accuracy*100:.4f}% \
                - val_loss: {val_loss:.4f} - val_accuracy: {val_accuracy*100:.4f}%')

    #train in one epoch, return the train_acc, train_loss
    def train_one_epoch(self):
        self.model.train()
        running_loss, correct, total = 0.0, 0, 0
        for batch in self.train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
            self.optimizer.zero_grad()
            outputs = self.model(input_ids= input_ids, attention_mask= attention_mask)
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_accuracy = correct / total
        train_loss = running_loss / self.num_batches
        return train_loss, train_accuracy

    #evaluate on a loader and return the loss and accuracy
    def evaluate(self, loader):
        self.model.eval()
        loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for batch in loader:
                input_ids = batch["input_ids"].to(device)
                labels = batch["label"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                outputs = self.model(input_ids= input_ids, attention_mask= attention_mask)
                loss = self.criterion(outputs, labels)
                loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        loss = loss / len(self.val_loader)
        return loss, accuracy

    #return the val_acc, val_loss, be called at the end of each epoch
    def validate_one_epoch(self):
      val_loss, val_accuracy = self.evaluate(self.val_loader)
      return val_loss, val_accuracy

We declare and train the prefix-prompt tuning model. In addition, you need to be patient with this model because it might converge slowly with many epochs.

In [41]:
import torch
# Automatically select the available device (MPS for Apple, CUDA for GPU, or CPU)
device = torch.device(
    "mps" if torch.backends.mps.is_available() else
    "cuda" if torch.cuda.is_available() else
    "cpu")

In [42]:
prefix_tuning_model = PrefixTuningForClassification(model_name = "bert-base-uncased", prefix_length = 5, data_manager = dm).to(device)

In [43]:
if prefix_tuning_model.prefix_length is not None:
  optimizer = torch.optim.Adam(list(prefix_tuning_model.classifier.parameters()) + [prefix_tuning_model.prefix_embeddings], lr=5e-5)
else:
  optimizer = torch.optim.Adam(prefix_tuning_model.classifier.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
trainer = FineTunedBaseTrainer(model= prefix_tuning_model, criterion=criterion, optimizer=optimizer, train_loader=train_loader, val_loader=valid_loader)
trainer.fit(num_epochs=100)

Epoch 1/100
26/26 - train_loss: 1.7779 - train_accuracy: 18.6758%                 - val_loss: 0.8483 - val_accuracy: 22.7273%
Epoch 2/100
26/26 - train_loss: 1.7416 - train_accuracy: 25.9838%                 - val_loss: 0.8281 - val_accuracy: 27.2727%
Epoch 3/100
26/26 - train_loss: 1.6979 - train_accuracy: 28.7945%                 - val_loss: 0.8088 - val_accuracy: 35.3535%
Epoch 4/100
26/26 - train_loss: 1.6691 - train_accuracy: 32.2299%                 - val_loss: 0.7881 - val_accuracy: 38.8889%
Epoch 5/100
26/26 - train_loss: 1.6408 - train_accuracy: 33.8538%                 - val_loss: 0.7713 - val_accuracy: 41.4141%
Epoch 6/100
26/26 - train_loss: 1.6171 - train_accuracy: 37.2892%                 - val_loss: 0.7629 - val_accuracy: 44.9495%
Epoch 7/100
26/26 - train_loss: 1.5975 - train_accuracy: 41.9113%                 - val_loss: 0.7554 - val_accuracy: 47.4747%
Epoch 8/100
26/26 - train_loss: 1.5891 - train_accuracy: 45.4716%                 - val_loss: 0.7429 - val_accuracy: 5

#### <font color="red">**Question 4.3**</font>
**For any models defined in the previous questions (of all parts), you are free to fine-tune hyperparameters, e.g., `optimizer`, `learning_rate`, `state_sizes`, such that you get a best model, i.e., the one with the highest accuracy on the test set. You will need to report (i) what is your best model,  (ii) its accuracy on the test set, and (iii) the values of its hyperparameters. Note that you must report your best model's accuracy with rounding to 4 decimal places, i.e., 0.xxxx. You will also need to upload your best model (or provide us with the link to download your best model). The assessment will be based on your best model's accuracy, with up to 10 marks available, specifically:**
* The best accuracy $\ge$ 0.97: 10 marks
* 0.97 $>$ The best accuracy $\ge$ 0.92: 7 marks
* 0.92 $>$ The best accuracy $\ge$ 0.85: 4 marks
* The best accuracy $<$ 0.85: 0 mark

**For this question, you can put below the code to train the best model. In this case, you need to show your code and the evidence of running regarding the best model. Moreover, if you save the best model, you need to provide the link to download the best model, the code to load the best model, and then evaluate on the test set.**
<div style="text-align: right"><font color="red">[10 marks]</font></div>

# Give your answer here.

(i) What is your best model?

Based on the highest test set accuracy achieved across all executed models in the provided notebooks, the best performance was obtained by multiple models:
* TransformerClassifier (Question 4.1)
* RNN (GRU/Scratch Mode) (Question 3.2.2)
* TextCNN (Question 2)
I select the $\mathbf{TransformerClassifier}$ from the final notebook for the detailed hyperparameter report.


(ii) The accuracy of your best model on the test set


The test set accuracy for the selected model, rounded to 4 decimal places, is:$$\mathbf{0.9751}$$This accuracy ($\ge 0.97$) corresponds to the 10 marks category according to Question 4.3 instructions.


(iii) The values of the hyperparameters of your best model


The selected best model is the TransformerClassifier trained in Question 4.1. The hyperparameters are:
$$
\begin{array}{|l|l|}
\hline
\mathbf{Hyperparameter} & \mathbf{Value} \\
\hline
\text{Model Class} & \text{TransformerClassifier} \\
\hline
\text{Architecture} & \text{Transformer Encoder with Mean Pooling} \\
\hline
\mathbf{embed\_dim} & 512 \\
\hline
\mathbf{num\_heads} & 8 \\
\hline
\mathbf{ff\_dim} & 2048 \\
\hline
\mathbf{num\_layers} & 6 \\
\hline
\mathbf{dropout\_rate} & 0.1 \\
\hline
\text{Optimizer} & \text{Adam} \\
\hline
\mathbf{lr} & 1 \times 10^{-4} \\
\hline
\text{Epochs} & 30 \\
\hline
\end{array}
$$


(iv) The link to download your best model

https://drive.google.com/file/d/13zMZTwUeWl_06Qy0_8k31ono8_jMNZj7/view?usp=drive_link

---
<div style="text-align: center"> <font color="green">GOOD LUCK WITH YOUR ASSIGNMENT 2!</font> </div>
<div style="text-align: center"> <font color="black">END OF ASSIGNMENT</font> </div>

In [48]:
!jupyter nbconvert --to html "FIT5215_DeepLearning_Assignment2_Official[Transformers].ipynb"

[NbConvertApp] WARNING | pattern 'FIT5215_DeepLearning_Assignment2_Official[Transformers].ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: 